<a href="https://colab.research.google.com/github/AllanRodrigo/Data_Science_Alura/blob/main/Notebooks/Semana_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Preparando o Ambiente**

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 19.5 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=af332859eafcdbf41c514889b0871e0d320cc9cd64ee931ba93d53d8a70ccedf
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Regressão com Spark") \
    .getOrCreate()

spark

## **Preços de Imóveis - Transformados**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import zipfile

zipfile.ZipFile('/content/drive/MyDrive/curso-spark/semana-2.zip', 'r').extractall('/content/drive/MyDrive/curso-spark/semana_2')

In [99]:
path = '/content/drive/MyDrive/curso-spark/semana_2/dataset_transformado_parquet'
dados = spark.read.parquet(path)

In [50]:
dados.show(10)

+--------------------+-----+----------+---------+---------+--------------------+-------+------+------------+------------+-----------+----+--------------------+------------+----------+----+-----+------+
|                  id|andar|area_total|area_util|banheiros|     caracteristicas|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|              bairro|        zona|condominio|iptu| tipo| valor|
+--------------------+-----+----------+---------+---------+--------------------+-------+------+------------+------------+-----------+----+--------------------+------------+----------+----+-----+------+
|03a386b6-7ab8-4ef...|    0|        43|       43|        1|[Churrasqueira, A...|      2|  null|       Usado| Apartamento|Residencial|   1|            Realengo|  Zona Oeste|       285|null|Venda| 22999|
|1fe78d41-b8e0-4d2...|    0|        44|       44|        1|                  []|      2|     0|       Usado| Apartamento|Residencial|   0|               Irajá|  Zona Norte|       170|   0|Vend

## **Seleção de features**

In [51]:
dados.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: long (nullable = true)
 |-- area_total: string (nullable = true)
 |-- area_util: string (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: long (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: string (nullable = true)
 |-- iptu: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: string (nullable = true)



In [52]:
from pyspark.sql.types import DoubleType, StringType, IntegerType
from pyspark.sql import functions as f

In [100]:
dados\
    .select('tipo_anuncio')\
    .groupBy('tipo_anuncio')\
    .count()\
    .show()

+------------+-----+
|tipo_anuncio|count|
+------------+-----+
|       Usado|66562|
+------------+-----+



In [54]:
dados\
    .select('tipo_unidade')\
    .groupBy('tipo_unidade')\
    .count()\
    .show()

+------------+-----+
|tipo_unidade|count|
+------------+-----+
| Apartamento|66562|
+------------+-----+



In [55]:
dados\
    .select('tipo_uso')\
    .groupBy('tipo_uso')\
    .count()\
    .show()

+-----------+-----+
|   tipo_uso|count|
+-----------+-----+
|Residencial|66562|
+-----------+-----+



In [56]:
dados\
    .select('tipo')\
    .groupBy('tipo')\
    .count()\
    .show()

+-----+-----+
| tipo|count|
+-----+-----+
|Venda|66562|
+-----+-----+



In [101]:
dados = dados.select("id", "andar", "area_util", "banheiros", "caracteristicas", "quartos", \
             "suites", "vaga", "bairro", "zona", "condominio", "iptu", "valor")

## **Converter os tipos das colunas**

In [102]:
dados = dados.withColumn('area_util', dados['area_util'].cast(DoubleType()))\
                   .withColumn('condominio', dados['condominio'].cast(DoubleType()))\
                   .withColumn('iptu', dados['iptu'].cast(DoubleType()))\
                   .withColumn('valor', dados['valor'].cast(DoubleType()))\
                   .withColumn('andar', dados['andar'].cast(IntegerType()))\
                   .withColumn('banheiros', dados['banheiros'].cast(IntegerType()))\
                   .withColumn('quartos', dados['quartos'].cast(IntegerType()))\
                   .withColumn('suites', dados['suites'].cast(IntegerType()))

dados.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: integer (nullable = true)
 |-- area_util: double (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- quartos: integer (nullable = true)
 |-- suites: integer (nullable = true)
 |-- vaga: long (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: double (nullable = true)
 |-- iptu: double (nullable = true)
 |-- valor: double (nullable = true)



## **Tratamento da coluna caracteristicas**

In [103]:
dados = dados.withColumn('caracteristicas', f.when((f.size(f.col('caracteristicas')) == 0), f.lit(None)).otherwise(f.col('caracteristicas')))


## **Tratar dados nulos e NaN's**

In [104]:
colunas = ["id", "andar", "area_util", "banheiros", "quartos", \
             "suites", "vaga", "bairro", "zona", "condominio", "iptu", "valor"]

dados\
    .select([f.count(f.when(f.isnan(c) | f.isnull(c), True)).alias(c) for c in colunas])\
    .show()

+---+-----+---------+---------+-------+------+----+------+----+----------+----+-----+
| id|andar|area_util|banheiros|quartos|suites|vaga|bairro|zona|condominio|iptu|valor|
+---+-----+---------+---------+-------+------+----+------+----+----------+----+-----+
|  0|    0|        0|        0|      0|  5554|3017|     0|   0|      2371|7199|    0|
+---+-----+---------+---------+-------+------+----+------+----+----------+----+-----+



In [105]:
dados = dados\
    .select('*')\
    .na\
    .fill(0)

In [93]:
dados\
    .select([f.count(f.when(f.isnan(c) | f.isnull(c), True)).alias(c) for c in colunas])\
    .show()

+---+-----+---------+---------+-------+------+----+------+----+----------+----+-----+
| id|andar|area_util|banheiros|quartos|suites|vaga|bairro|zona|condominio|iptu|valor|
+---+-----+---------+---------+-------+------+----+------+----+----------+----+-----+
|  0|    0|        0|        0|      0|     0|   0|     0|   0|         0|   0|    0|
+---+-----+---------+---------+-------+------+----+------+----+----------+----+-----+



## **Preparação dos dados para machine learning**

In [77]:
dados\
    .select('zona')\
    .groupBy('zona')\
    .count()\
    .show()

+------------+-----+
|        zona|count|
+------------+-----+
|  Zona Norte|11897|
|  Zona Oeste|32979|
|Zona Central| 1144|
|    Zona Sul|20531|
|            |   11|
+------------+-----+



In [106]:
dados = dados\
    .where(f.col('zona') != '')

In [107]:
zona = dados\
    .groupBy('id')\
    .pivot('zona')\
    .agg(f.lit(1))\
    .na\
    .fill(0)

zona.show(5)

+--------------------+------------+----------+----------+--------+
|                  id|Zona Central|Zona Norte|Zona Oeste|Zona Sul|
+--------------------+------------+----------+----------+--------+
|4e47e4d4-3326-4eb...|           0|         0|         0|       1|
|02fba6ef-a691-442...|           0|         0|         1|       0|
|fc03c1a9-8bbb-41a...|           0|         1|         0|       0|
|3dd5d200-0a7f-43d...|           0|         0|         0|       1|
|82707939-71bd-40c...|           0|         0|         0|       1|
+--------------------+------------+----------+----------+--------+
only showing top 5 rows



In [108]:
caracteristicas = dados.select(dados.id,f.explode(dados.caracteristicas))
caracteristicas.groupBy('col').count().show()

+------------------+-----+
|               col|count|
+------------------+-----+
|Condomínio fechado|34980|
|        Playground|31854|
| Portão eletrônico|29323|
|           Piscina|33235|
|Animais permitidos|30617|
|      Portaria 24h|30360|
|          Elevador|43023|
|          Academia|27641|
|   Salão de festas|33582|
|     Churrasqueira|31737|
+------------------+-----+



In [109]:
caracteristicas = caracteristicas\
                .groupBy('id')\
                .pivot('col')\
                .agg(f.lit(1))\
                .na\
                .fill(0)
caracteristicas.show()

+--------------------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|                  id|Academia|Animais permitidos|Churrasqueira|Condomínio fechado|Elevador|Piscina|Playground|Portaria 24h|Portão eletrônico|Salão de festas|
+--------------------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|fd96bbd5-d631-416...|       1|                 1|            1|                 1|       1|      1|         1|           1|                1|              1|
|bfffedfe-99e7-4ae...|       0|                 1|            1|                 1|       1|      0|         1|           0|                1|              1|
|fcb67af3-5601-415...|       1|                 1|            1|                 0|       0|      1|         1|           0|                0|              1|
|afecddff-f4cc-4ab...|       1|               

In [110]:
dados = dados\
    .join(zona, 'id', how='inner')\
    .join(caracteristicas, 'id', how='left')
    #.drop('caracteristicas')\
    #.drop('zona')

# Limpar os dados nulos gerados pelo left join de caracteristicas
dados = dados\
    .select('*')\
    .na\
    .fill(0)

dados.show()

+--------------------+-----+---------+---------+--------------------+-------+------+----+--------------------+------------+----------+-------+---------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|                  id|andar|area_util|banheiros|     caracteristicas|quartos|suites|vaga|              bairro|        zona|condominio|   iptu|    valor|Zona Central|Zona Norte|Zona Oeste|Zona Sul|Academia|Animais permitidos|Churrasqueira|Condomínio fechado|Elevador|Piscina|Playground|Portaria 24h|Portão eletrônico|Salão de festas|
+--------------------+-----+---------+---------+--------------------+-------+------+----+--------------------+------------+----------+-------+---------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|

## **Salvar a base de dados em parquet**

In [111]:
dados.write.parquet(
    path='/content/drive/MyDrive/curso-spark/semana_2/modelo',
    mode='overwrite'
)